In [1]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
os.environ["CORENLP_HOME"] = "/home/jinming/PhD/2nd_Project_GameAgents_pycharm/third-packages/stanford-corenlp-4.5.7"
print(os.getenv("CORENLP_HOME"))

/home/jinming/PhD/2nd_Project_GameAgents_pycharm/third-packages/stanford-corenlp-4.5.7


**Dependency Parse**: This will show relations like nsubj, prep, pobj, amod, which will be crucial in forming triples.

**Named Entities**: The NER output will show named entities (e.g., LOCATION, ORGANIZATION) that can be helpful for identifying subjects and objects.

In [2]:
%%markdown
from stanfordnlp.server import CoreNLPClient

# The observation text
observation_text = """
You are standing in an open field west of a white house, with a boarded front door. 
There is a small mailbox here.
"""

# Set up the CoreNLP client
with CoreNLPClient(annotators=['tokenize', 'ssplit', 'pos', 'lemma', 'ner', 'depparse'], timeout=30000, memory='4G') as client:
    
    # Annotate the text
    ann = client.annotate(observation_text)
    
    # Iterate through sentences
    for sentence in ann.sentence:
        # Print dependency parsing information
        print("Dependency Parse:")
        for dep in sentence.basicDependencies.edge:
            dep_relation = dep.dep
            governor = sentence.token[dep.source - 1].word  # source token (index is 1-based)
            dependent = sentence.token[dep.target - 1].word  # target token
            print(f"{governor} ({dep_relation}) -> {dependent}")
        
        # Print Named Entities for entity extraction
        print("\nNamed Entities:")
        for token in sentence.token:
            word = token.word
            ner = token.ner
            print(f"{word} -> {ner}")


from stanfordnlp.server import CoreNLPClient

# The observation text
observation_text = """
You are standing in an open field west of a white house, with a boarded front door. 
There is a small mailbox here.
"""

# Set up the CoreNLP client
with CoreNLPClient(annotators=['tokenize', 'ssplit', 'pos', 'lemma', 'ner', 'depparse'], timeout=30000, memory='4G') as client:
    
    # Annotate the text
    ann = client.annotate(observation_text)
    
    # Iterate through sentences
    for sentence in ann.sentence:
        # Print dependency parsing information
        print("Dependency Parse:")
        for dep in sentence.basicDependencies.edge:
            dep_relation = dep.dep
            governor = sentence.token[dep.source - 1].word  # source token (index is 1-based)
            dependent = sentence.token[dep.target - 1].word  # target token
            print(f"{governor} ({dep_relation}) -> {dependent}")
        
        # Print Named Entities for entity extraction
        print("\nNamed Entities:")
        for token in sentence.token:
            word = token.word
            ner = token.ner
            print(f"{word} -> {ner}")


In [3]:
%%markdown
from stanfordnlp.server import CoreNLPClient

# The observation text
observation_text = """
West of House
You are standing in an open field west of a white house, with a boarded front door.There is a small mailbox here.
"""

# Start the CoreNLP Client
with CoreNLPClient(annotators=['tokenize', 'ssplit', 'pos', 'lemma', 'ner', 'depparse'], timeout=30000, memory='4G') as client:
    
    # Annotate the text
    ann = client.annotate(observation_text)

    # Iterate through sentences and print the dependency parse
    for sentence in ann.sentence:
        print("\nSentence:", " ".join([token.word for token in sentence.token]))
        print("Dependency Parse:")
        for dep in sentence.basicDependencies.edge:
            dep_relation = dep.dep
            governor = sentence.token[dep.source - 1].word  # source token (index is 1-based)
            dependent = sentence.token[dep.target - 1].word  # target token
            print(f"{governor} ({dep_relation}) -> {dependent}")


from stanfordnlp.server import CoreNLPClient

# The observation text
observation_text = """
West of House
You are standing in an open field west of a white house, with a boarded front door.There is a small mailbox here.
"""

# Start the CoreNLP Client
with CoreNLPClient(annotators=['tokenize', 'ssplit', 'pos', 'lemma', 'ner', 'depparse'], timeout=30000, memory='4G') as client:
    
    # Annotate the text
    ann = client.annotate(observation_text)

    # Iterate through sentences and print the dependency parse
    for sentence in ann.sentence:
        print("\nSentence:", " ".join([token.word for token in sentence.token]))
        print("Dependency Parse:")
        for dep in sentence.basicDependencies.edge:
            dep_relation = dep.dep
            governor = sentence.token[dep.source - 1].word  # source token (index is 1-based)
            dependent = sentence.token[dep.target - 1].word  # target token
            print(f"{governor} ({dep_relation}) -> {dependent}")


In [3]:
from fuzzywuzzy import fuzz
import logging
import json
import requests
from jericho.util import clean

def call_stanford_openie(sentence):
    url = "http://localhost:9000/"
    querystring = {
        "properties": "%7B%22annotators%22%3A%20%22openie%22%7D",
        "pipelineLanguage": "en"}
    try:
        response = requests.request("POST", url, data=sentence, params=querystring)
        response = json.JSONDecoder().decode(response.text)
    except:
        print("OpenIE error")
        logging.debug('OpenIE Error: ' + sentence)
        response = {"sentences": ""}
    return response


visible_state = """West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.
"""

inventory_state = """

"""
prev_action = None
# obj is provided by Jericho api function .__identify_interactive_objects()
objs = {'The small mailbox is closed.': [('mailbox', 'NOUN', 'LOC'), ('small', 'ADJ', 'LOC')], 'The door is closed.': [('front', 'ADJ', 'LOC'), ('door', 'NOUN', 'LOC')], 'The house is a beautiful colonial house which is painted white. It is clear that the owners must have been extremely wealthy.': [('house', 'NOUN', 'LOC'), ('white', 'NOUN', 'LOC')], 'all': [('all', 'NOUN', 'LOC')]}


visible_state = visible_state.split('\n')
room = visible_state[0]
print(visible_state)
visible_state = clean(' '.join(visible_state[1:]))

dirs = ['north', 'south', 'east', 'west', 'southeast', 'southwest', 'northeast', 'northwest', 'up', 'down']

visible_state = str(visible_state)
rules = []

sents = call_stanford_openie(visible_state)['sentences']

in_aliases = ['are in', 'are facing', 'are standing', 'are behind', 'are above', 'are below', 'are in front']

in_rl = []
in_flag = False

for i, ov in enumerate(sents):
    sent = ' '.join([a['word'] for a in ov['tokens']])
    triple = ov['openie']
    for d in dirs:
        if d in sent and i != 0:
            rules.append((room, 'has', 'exit to ' + d))

    for tr in triple:
        h, r, t = tr['subject'].lower(), tr['relation'].lower(), tr['object'].lower()
        if h == 'you':
            for rp in in_aliases:
                if fuzz.token_set_ratio(r, rp) > 80:
                    r = "in"
                    in_rl.append((h, r, t))
                    in_flag = True
                    break

        if h == 'it':
            break
        if not in_flag:
            rules.append((h, r, t))

if in_flag:
    cur_t = in_rl[0]
    for h, r, t in in_rl:
        if set(cur_t[2].split()).issubset(set(t.split())):
            cur_t = h, r, t
    rules.append(cur_t)
    room = cur_t[2]

try:
    items = inventory_state.split(':')[1].split('\n')[1:]
    for item in items:
        rules.append(('you', 'have', str(' ' .join(item.split()[1:]))))
except:
    pass

if prev_action is not None:
    pass
    # for d in dirs:
    #     if d in prev_action and self.room != "":
    #         rules.append((prev_room, d + ' of', room))
    #         if prev_room_subgraph is not None:
    #             for ed in prev_room_subgraph.edges:
    #                 rules.append((ed[0], prev_room_subgraph[ed]['rel'], ed[1]))
    #         break

for o in objs:
    #if o != 'all':
    rules.append((str(o), 'in', room))

add_rules = rules




['West of House', 'You are standing in an open field west of a white house, with a boarded front door.', 'There is a small mailbox here.', '']
OpenIE error


In [4]:
print(add_rules)
print('*'*60)

# for sent in sents:
#     for dep in sent['basicDependencies']:
#         print(dep)
#         print('-' * 60)
#     print('='*60)

[('The small mailbox is closed.', 'in', 'West of House'), ('The door is closed.', 'in', 'West of House'), ('The house is a beautiful colonial house which is painted white. It is clear that the owners must have been extremely wealthy.', 'in', 'West of House'), ('all', 'in', 'West of House')]
************************************************************
